In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
import os
from PIL import Image

class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layers with dropout
        self.fc1 = nn.Linear(32 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)  # Output is a single value (real or generated)
        # Dropout layer for regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutional and pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the output for the fully connected layers
        x = torch.flatten(x, 1)
        # Apply fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for binary classification
        return x

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert("RGB")  # Convert to RGB in case of grayscale
        if self.transform:
            image = self.transform(image)
        label = 0 if "RealArt" in self.root_dir else 1  # RealArt: 0, AiArtData: 1
        return image, label

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.RandomHorizontalFlip(),  # Augmentation: Random horizontal flip
    transforms.RandomRotation(15),       # Augmentation: Random rotation by 15 degrees
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),
])
# Load datasets
real_dataset = CustomDataset('/kaggle/input/ai-generated-images-vs-real-images/RealArt/RealArt/', transform=transform)
generated_dataset = CustomDataset('/kaggle/input/ai-generated-images-vs-real-images/AiArtData/AiArtData/', transform=transform)

# Split datasets into train, validation, and test
real_train, real_val_test = train_test_split(real_dataset, test_size=0.2, random_state=45)
gen_train, gen_val_test = train_test_split(generated_dataset, test_size=0.2, random_state=45)

real_val, real_test = train_test_split(real_val_test, test_size=0.5, random_state=45)
gen_val, gen_test = train_test_split(gen_val_test, test_size=0.5, random_state=42)

# Combine datasets for training, validation, and test
train_dataset = torch.utils.data.ConcatDataset([real_train, gen_train])
val_dataset = torch.utils.data.ConcatDataset([real_val, gen_val])
test_dataset = torch.utils.data.ConcatDataset([real_test, gen_test])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Initialize model, loss function, and optimizer
model = ImageClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0002)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [25]:
import time
import torch
import torch.optim as optim

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_val_loss = float('inf')  # Initialize with a very large number
best_model_path = "best_model.pth"  # Path to save the best model

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()

    # Calculate average losses
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # Save the model if validation loss has improved
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"  Saved best model with val loss: {best_val_loss:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training complete. Elapsed time: {elapsed_time:.2f} seconds")

Epoch 1/10, Train Loss: 0.6971, Val Loss: 0.6310
  Saved best model with val loss: 0.6310
Epoch 2/10, Train Loss: 0.6794, Val Loss: 0.6119
  Saved best model with val loss: 0.6119
Epoch 3/10, Train Loss: 0.6630, Val Loss: 0.6217
Epoch 4/10, Train Loss: 0.6588, Val Loss: 0.5963
  Saved best model with val loss: 0.5963
Epoch 5/10, Train Loss: 0.6465, Val Loss: 0.5774
  Saved best model with val loss: 0.5774
Epoch 6/10, Train Loss: 0.6323, Val Loss: 0.6261
Epoch 7/10, Train Loss: 0.6261, Val Loss: 0.5581
  Saved best model with val loss: 0.5581
Epoch 8/10, Train Loss: 0.5786, Val Loss: 0.5793
Epoch 9/10, Train Loss: 0.5630, Val Loss: 0.5845
Epoch 10/10, Train Loss: 0.5004, Val Loss: 0.6592
Training complete. Elapsed time: 7.13 seconds


In [26]:
import numpy as np
from sklearn.metrics import classification_report
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        predicted_labels.extend(predicted.cpu().numpy().flatten())
        true_labels.extend(labels.cpu().numpy().flatten())

predicted_labels = np.array(predicted_labels).astype(int)
true_labels = np.array(true_labels).astype(int)

# Compute and print classification report
report = classification_report(true_labels, predicted_labels, target_names=["Real", "Generated"], digits=4)
print(report)

              precision    recall  f1-score   support

        Real     0.0000    0.0000    0.0000        44
   Generated     0.5510    1.0000    0.7105        54

    accuracy                         0.5510        98
   macro avg     0.2755    0.5000    0.3553        98
weighted avg     0.3036    0.5510    0.3915        98



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
